In [2]:
# Imports
import pandas as pd
import numpy as np
import polars as pl
import os
import awpy
from demoparser2 import DemoParser
from awpy import Demo
from pathlib import Path
from awpy.stats import adr
from awpy.stats import kast
from awpy.stats import rating
from awpy.stats import calculate_trades

folder_path = r'C:\Users\bayli\Documents\CS Demos\All_IEM_Katowice_2025\IEM_Katowice_2025'
file_path = r'C:\Users\bayli\Documents\Git Projects\test_demos\natus-vincere-vs-faze-m1-inferno.dem'

# Creating DataFrames
df_flashes = pd.DataFrame()
df_he = pd.DataFrame()
df_infernos = pd.DataFrame()
df_smoke = pd.DataFrame()
df_kills = pd.DataFrame()
df_rounds = pd.DataFrame()
df_all_first_kills = pd.DataFrame()
df_adr = pd.DataFrame()
df_kast = pd.DataFrame()
team_rounds_won = pd.DataFrame()
players_id = pd.DataFrame()
i = 1

In [ ]:
dem = Demo(file_path)
dem.parse(player_props=["team_clan_name","total_rounds_played"])

In [4]:
player_hurt = dem.events.get('player_hurt', pl.DataFrame())
if player_hurt is not None and len(player_hurt) > 0:
    player_hurt = pd.concat([df_flashes, player_hurt.to_pandas()], ignore_index=True)

In [6]:
util_dmg = player_hurt[(player_hurt["weapon"] == "hegrenade") | (player_hurt["weapon"] == "molotov") | (player_hurt["weapon"] == "inferno")]

In [7]:
util_dmg.to_csv('util_dmg.csv')

In [ ]:
util_dmg.loc[:, "total_dmg"] = util_dmg["dmg_armor"] + util_dmg["dmg_health"]

agg_dmg = util_dmg.groupby("attacker_steamid").agg(
    total_dmg=("total_dmg", "sum"),
    ct_total_dmg=("total_dmg", lambda x: x[util_dmg.loc[x.index, "attacker_side"] == "ct"].sum()),
    t_total_dmg=("total_dmg", lambda x: x[util_dmg.loc[x.index, "attacker_side"] == "t"].sum())
).reset_index()

agg_dmg